In [4]:
!git clone https://github.com/bowman-aikium/aikium_thermal_stability
%cd aikium_thermal_stability

Cloning into 'aikium_thermal_stability'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 68 (delta 19), reused 58 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 3.24 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/aikium_thermal_stability/aikium_thermal_stability


In [5]:
!pip install sympy==1.12
import sympy
print(sympy.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 52.6 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.8.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.
1.12


In [6]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from google.colab import files



# Import from your Model folder
from Model.model import thermalMLP, ProteinTmDataset


In [11]:
embedding_np = np.load("/content/esm2_embeddings_with_ids.npz", allow_pickle=True)
dataset_df = pd.read_csv("/content/Stratified_data.csv")

embedding = embedding_np["embeddings"]   # shape: (N, hidden_dim)
ids = embedding_np["ids"]

emb_df = pd.DataFrame({
    "ID": ids,
    "embedding": list(embedding)
})

emb_df["ID"] = emb_df["ID"].astype(int)
dataset_df["ID"] = dataset_df["ID"].astype(int)
merged = emb_df.merge(dataset_df, on="ID")
merged




,ID,embedding,Tm,Sequence,From,AA_length,MW
0,117012,"[3.4952238, 2.9369724, -3.406106, -1.0034976, ...",45.556042,MKWAYKEENNFEKRRAEGDKIRRKYPDRIPVIVEKAPKSKLHDLDK...,meltome,123,14.763731
1,313786,"[2.2217994, -2.0552912, -0.14634663, 1.3754547...",43.863329,MVKATNVDLSLEDIISKTRKTTGSIQKKSFGGARRGNTRPTGLPRR...,meltome,240,25.695370
2,169171,"[3.0030677, 4.205013, 4.074065, 0.7811, -6.895...",43.574470,MPRANEIKKGMVLNYNGKLLLVKDIDIQSPTARGAATLYKMRFSDV...,meltome,190,21.532381
3,227821,"[4.7244706, -0.123155594, 1.057647, 4.5875287,...",40.496359,MTKSELIERMLTKQPQLSAKDVELAVKTILDHMSQSLSTGERIEIR...,meltome,96,10.757199
4,97927,"[2.16275, -1.2902148, 2.1692014, 0.009807652, ...",37.073883,MRQVVLDTETTGIGAEKGHRIIEIGCVELIDRKLTGRHYHQYVNPQ...,meltome,234,26.063256
...,...,...,...,...,...,...,...
23415,200713,"[3.211512, 0.9198779, -0.08337058, -1.2356853,...",86.289547,MTVRQVLVHKGGGVHAIHPEATVLDALRKLAEHDIGALLVMEGERL...,meltome,143,15.916415
23416,267005,"[-1.4136505, -2.4800174, 0.06798242, 2.689519,...",88.057949,MTGLELLAVALGMRHGVDPDHLAAVDGLSRVRPSPLNGVLFALGHG...,meltome,220,22.874996
23417,269836,"[2.5266905, -5.975683, -3.1002605, 1.2522675, ...",88.583454,MASLSFMIKEYNDYYIIDFERPVRKFSSAPFNGGVGTSLRYINRTV...,meltome,215,24.262483
23418,298355,"[0.30273947, 2.1589444, -0.4573475, -0.4192088...",86.755190,MRFKAELMNAPEMRRALYRIAHEIVEANKGTEGLALVGIHTRGIPL...,meltome,181,20.466363


In [10]:
def train_and_eval(params, input_dim, train_loader, val_loader):

    model = thermalMLP(
        input_dim=input_dim,
        hidden_sizes=params["hidden_sizes"],
        activation=params["activation"],
        dropout=params["dropout"]
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params["lr"])

    train_losses = []
    val_losses = []

    for epoch in range(params["epochs"]):
        # ---- Training ----
        model.train()
        epoch_train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(train_loss)

        # ---- Validation ----
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                epoch_val_loss += criterion(preds, yb).item()
        val_loss = epoch_val_loss / len(val_loader)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{params['epochs']} | "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    return train_losses, val_losses

In [12]:
import itertools

# Define parameter grid
activation_list = [nn.ReLU, nn.GELU, nn.SiLU]
dropout_list = [0.0]
lr_list = [1e-3, 5e-4, 1e-4]


# Build full grid
hyperparam_sweep = []
for a, d, lr in itertools.product(activation_list, dropout_list, lr_list):
    hyperparam_sweep.append({
        "hidden_sizes": [1024, 512, 256],
        "activation": a,
        "dropout": d,
        "lr": lr,
        "epochs": 50
    })

print(f"Total configs: {len(hyperparam_sweep)}")

Total configs: 9


In [18]:
embeddings = np.stack(merged["embedding"].values)
Tm = merged["Tm"].values
input_dim = embeddings.shape[1]

X_train, X_val, y_train, y_val = train_test_split(embeddings, Tm, test_size=0.2, random_state=42)

train_ds = ProteinTmDataset(X_train, y_train)
val_ds = ProteinTmDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size= 128 * 4, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=128 * 4)


results = []
for i, params in enumerate(hyperparam_sweep):
    train_loss, val_loss = train_and_eval(params, input_dim, train_loader, val_loader)
    results.append((params, train_loss, val_loss))
    plt.figure()
    plt.plot(train_loss, label="Train Loss")
    plt.plot(val_loss, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.title(f'size:{params["hidden_sizes"]}, activation: {params["activation"]}, lr:{params["lr"]}', fontsize = "smaller")
    plt.legend()

    fig_filename = f"loss_plot_config{i+1}_tuning-09152025.png"
    plt.savefig(fig_filename)
    plt.close()

    files.download(fig_filename)

# Show best
best = sorted(results, key=lambda x: x[2][-1])[0]
print("\nBest config:", best[0], "| Final Val Loss:", best[2][-1])

Epoch 1/50 | Train Loss: 517.1840 | Val Loss: 105.7762
Epoch 2/50 | Train Loss: 93.1846 | Val Loss: 86.7584
Epoch 3/50 | Train Loss: 77.8388 | Val Loss: 78.6329
Epoch 4/50 | Train Loss: 65.6251 | Val Loss: 64.6668
Epoch 5/50 | Train Loss: 56.1565 | Val Loss: 56.3812
Epoch 6/50 | Train Loss: 46.2781 | Val Loss: 48.1103
Epoch 7/50 | Train Loss: 38.8511 | Val Loss: 42.9669
Epoch 8/50 | Train Loss: 31.8038 | Val Loss: 36.7530
Epoch 9/50 | Train Loss: 26.3834 | Val Loss: 30.3336
Epoch 10/50 | Train Loss: 23.5374 | Val Loss: 26.3812
Epoch 11/50 | Train Loss: 19.7147 | Val Loss: 24.4198
Epoch 12/50 | Train Loss: 17.0647 | Val Loss: 23.2927
Epoch 13/50 | Train Loss: 15.3867 | Val Loss: 22.2814
Epoch 14/50 | Train Loss: 13.7871 | Val Loss: 19.5538
Epoch 15/50 | Train Loss: 13.9718 | Val Loss: 18.4124
Epoch 16/50 | Train Loss: 11.6253 | Val Loss: 17.9472
Epoch 17/50 | Train Loss: 11.0814 | Val Loss: 16.8417
Epoch 18/50 | Train Loss: 12.6804 | Val Loss: 15.9289
Epoch 19/50 | Train Loss: 10.1322 |

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/50 | Train Loss: 701.9437 | Val Loss: 121.3907
Epoch 2/50 | Train Loss: 110.5452 | Val Loss: 96.1140
Epoch 3/50 | Train Loss: 83.6454 | Val Loss: 76.6523
Epoch 4/50 | Train Loss: 65.7130 | Val Loss: 63.9915
Epoch 5/50 | Train Loss: 54.3814 | Val Loss: 56.8766
Epoch 6/50 | Train Loss: 46.7858 | Val Loss: 49.7250
Epoch 7/50 | Train Loss: 41.6290 | Val Loss: 48.3845
Epoch 8/50 | Train Loss: 36.7896 | Val Loss: 41.0892
Epoch 9/50 | Train Loss: 32.1764 | Val Loss: 37.4931
Epoch 10/50 | Train Loss: 29.0212 | Val Loss: 35.6182
Epoch 11/50 | Train Loss: 26.4395 | Val Loss: 31.9708
Epoch 12/50 | Train Loss: 24.2787 | Val Loss: 29.3182
Epoch 13/50 | Train Loss: 21.8762 | Val Loss: 27.3366
Epoch 14/50 | Train Loss: 20.3664 | Val Loss: 26.0440
Epoch 15/50 | Train Loss: 18.6458 | Val Loss: 25.0619
Epoch 16/50 | Train Loss: 18.1137 | Val Loss: 23.5288
Epoch 17/50 | Train Loss: 16.5742 | Val Loss: 21.9362
Epoch 18/50 | Train Loss: 15.0768 | Val Loss: 21.5329
Epoch 19/50 | Train Loss: 14.3582 

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Epoch 1/50 | Train Loss: 1773.7289 | Val Loss: 241.5707
Epoch 2/50 | Train Loss: 162.4895 | Val Loss: 122.8933
Epoch 3/50 | Train Loss: 101.2520 | Val Loss: 88.3872
Epoch 4/50 | Train Loss: 77.4548 | Val Loss: 74.5428
Epoch 5/50 | Train Loss: 66.4179 | Val Loss: 67.9416
Epoch 6/50 | Train Loss: 59.3605 | Val Loss: 61.5390
Epoch 7/50 | Train Loss: 52.7762 | Val Loss: 55.8365
Epoch 8/50 | Train Loss: 46.8229 | Val Loss: 50.3578
Epoch 9/50 | Train Loss: 41.3632 | Val Loss: 45.0904
Epoch 10/50 | Train Loss: 36.2185 | Val Loss: 42.7851
Epoch 11/50 | Train Loss: 31.5399 | Val Loss: 36.2540
Epoch 12/50 | Train Loss: 27.9419 | Val Loss: 32.6945
Epoch 13/50 | Train Loss: 24.2003 | Val Loss: 29.9397
Epoch 14/50 | Train Loss: 21.8175 | Val Loss: 28.2522
Epoch 15/50 | Train Loss: 20.0357 | Val Loss: 26.4888
Epoch 16/50 | Train Loss: 18.4370 | Val Loss: 23.8253
Epoch 17/50 | Train Loss: 16.3082 | Val Loss: 23.0076
Epoch 18/50 | Train Loss: 15.5307 | Val Loss: 21.8905
Epoch 19/50 | Train Loss: 14.10

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Best config: {'hidden_sizes': [1024, 512, 256], 'activation': <class 'torch.nn.modules.activation.SiLU'>, 'dropout': 0.0, 'lr': 0.0005, 'epochs': 50} | Final Val Loss: 11.112767505645753
